In [106]:
import collections
import random
import simpy
from scipy import stats

VELOCIDAD_LLEGADA = 1
NUM_BOLETO = 40
TIEMPO_SIMULACION = 7*24*60

boletos_lugares={}

def viaje(env, bus):
    while True:
        mu, sigma = 6, 4 # media y desvio estandar
        normal = stats.norm(mu, sigma)  
        aleatorios = normal.rvs(1) 
        if aleatorios>=0:
            break
    yield env.timeout(aleatorios)
    print('el ', bus, 'llego a las ', env.now, 'a su destino')
    yield env.timeout(aleatorios)
    print('el ', bus, 'regreso a las ', env.now)


def viaje_antes(env,bus):
  print('el bus salio antes')
  viaje(env, bus)


def ventaBoletos(env, num_boletos, lugar, buses):
  with buses.contador.request() as asiento:
      resultado = yield asiento | agencia.sold_out[lugar]
      if asiento not in resultado:
        agencia.num_renegados[lugar] += 1
        return
      if agencia.num_boletos[lugar] < num_boletos:
        aleatorios=0
        while True:
          mu, sigma = 10, 2 # media y desvio estandar
          normal = stats.norm(mu, sigma)  
          aleatorios = normal.rvs(1) 
          if aleatorios>=0:
              break
        yield env.timeout(aleatorios) 
        return
      else:
        if lugar in list(boletos_lugares.keys()):
          a=boletos_lugares.get(lugar)
          a.append(num_boletos)
          boletos_lugares[lugar]=a
        else:
          boletos_lugares[lugar]=[num_boletos]
      agencia.num_boletos[lugar] -= num_boletos
      if agencia.num_boletos[lugar] < 2:
        bus='bus con destino ',lugar
        yield env.process(viaje(env,bus))
        agencia.sold_out[lugar].succeed()
        agencia.tiempo_agotado[lugar] = env.now
        agencia.num_boletos[lugar] = 0


def llegadaClientes(env, teatro, VELOCIDAD_LLEGADA):
  while True:
    yield env.timeout(VELOCIDAD_LLEGADA)
    bernoulli = stats.bernoulli(0.5)
    pGenero = bernoulli.rvs(1) 
    genero=''
    if pGenero == 1:
        genero='masculino'
    else:
        genero='femenino'
    
    edad=0
    while True:
        mu = 20 # parametro de forma 
        poisson = stats.poisson(mu) 
        edad=poisson.rvs(1)
        if edad>=18:
            break
    
    lugar = random.choices(teatro.lugares, teatro.probabilidad, k=1)
    num_boletos = random.randint(1, 6)
    # print('la persona con nombre "nombre" con genero ',genero,' y edad ',edad, 'compro ', num_boletos,' boletos para ', lugar)
    if teatro.num_boletos[lugar[0]]:
      env.process(ventaBoletos(env, num_boletos, lugar[0], teatro))


Agencia = collections.namedtuple('Teatro', 'contador, lugares, probabilidad, num_boletos, sold_out, tiempo_agotado, num_renegados')

print('Teatro Carlos Crespi - UPS')
env = simpy.Environment()

contador = simpy.Resource(env,capacity=1)
lugares = ['Quito', 'Guayaquil', 'Cuenca', 'Baños', 'Tena', 'Ibarra']
probabilidad=[0.6,0.5,0.4,0.3,0.2,0.1]
num_boletos = {lugar: NUM_BOLETO for lugar in lugares}
sold_out = {lugar: env.event() for lugar in lugares}
tiempo_agotado = {lugar: None for lugar in lugares}
num_renegados = {lugar: 0 for lugar in lugares}

agencia = Agencia(contador, lugares, probabilidad, num_boletos, sold_out, tiempo_agotado, num_renegados)
env.process(llegadaClientes(env, agencia, VELOCIDAD_LLEGADA))
env.run(until=TIEMPO_SIMULACION)


Teatro Carlos Crespi - UPS
el  ('bus con destino ', 'Guayaquil') llego a las  [34.61946729] a su destino
el  ('bus con destino ', 'Guayaquil') regreso a las  [37.23893458]
el  ('bus con destino ', 'Quito') llego a las  [60.11071184] a su destino
el  ('bus con destino ', 'Quito') regreso a las  [71.6704778]
el  ('bus con destino ', 'Cuenca') llego a las  [78.98799161] a su destino
el  ('bus con destino ', 'Cuenca') regreso a las  [86.30550543]
el  ('bus con destino ', 'Tena') llego a las  [130.31340443] a su destino
el  ('bus con destino ', 'Tena') regreso a las  [135.78231741]
el  ('bus con destino ', 'Baños') llego a las  [205.17501809] a su destino
el  ('bus con destino ', 'Baños') regreso a las  [211.78753881]
el  ('bus con destino ', 'Ibarra') llego a las  [214.40091961] a su destino
el  ('bus con destino ', 'Ibarra') regreso a las  [217.01430041]


In [107]:
# Analisis y resultados
for lugar in lugares:
  if agencia.sold_out[lugar]:
    print('Los boletos para: %s se agoto en el tiempo %.1f despues de salir a la venta' %(lugar, agencia.tiempo_agotado[lugar]))
    print('Numero de personas que salieron de la fila/renegados %s' %agencia.num_renegados[lugar])

Los boletos para: Quito se agoto en el tiempo 71.7 despues de salir a la venta
Numero de personas que salieron de la fila/renegados 10
Los boletos para: Guayaquil se agoto en el tiempo 37.2 despues de salir a la venta
Numero de personas que salieron de la fila/renegados 0
Los boletos para: Cuenca se agoto en el tiempo 86.3 despues de salir a la venta
Numero de personas que salieron de la fila/renegados 4
Los boletos para: Baños se agoto en el tiempo 211.8 despues de salir a la venta
Numero de personas que salieron de la fila/renegados 10
Los boletos para: Tena se agoto en el tiempo 135.8 despues de salir a la venta
Numero de personas que salieron de la fila/renegados 0
Los boletos para: Ibarra se agoto en el tiempo 217.0 despues de salir a la venta
Numero de personas que salieron de la fila/renegados 1


In [108]:
print(boletos_lugares)

{'Tena': [5, 1, 4, 1, 1, 1, 2, 6, 1, 4, 5, 3, 6], 'Ibarra': [2, 6, 1, 5, 2, 2, 1, 3, 2, 6, 4, 5], 'Quito': [1, 2, 1, 1, 5, 3, 5, 3, 5, 6, 3, 4], 'Baños': [1, 5, 4, 4, 2, 3, 5, 1, 2, 1, 5, 5, 1], 'Cuenca': [4, 6, 4, 3, 2, 6, 3, 1, 3, 3, 1, 4], 'Guayaquil': [6, 2, 1, 4, 6, 6, 6, 4, 5]}


Se puede decir que dentro de esta simulacion no existen muchas personas renegadas o que se quedaron sin boleto ademas esta simulacion nos  da a entender que los lugares que mas se vendiero son para Quito